# Emotion-to-Sound Experiment 

In [1]:
%pylab
import ipywidgets as widgets
from functools import partial
from IPython.display import display
import pandas, imp, OSC, threading
import pickle, time,socket

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
# first: select sound model 
#soundmodel = 'abstract'
soundmodel = 'vocal'
if (soundmodel == 'abstract'):
    portMax = 9000
else: portMax = 9001
    
sonmod = imp.load_source('sonmod', 'EmoSonics-soundmodels.py')
print "wait for SC to start..."
time.sleep(5); # wait until SC is started...
print "resume: establish OSC interfaces..."
"""
To run vocal model: 1. make sure the path in EmoSonics-soundmodels.py has the correct path point at the sclang. 
                    2. make sure vowel is installed in Quark
To run abstract mode: Make sure the Max program AbstractModel is in the same folder
"""

"""
Setup the OSC model here. 
"""

clientSC  = OSC.OSCClient(); clientSC.connect(("127.0.0.1", 57110)) # SC uses 57110 by default. 

def sc_msg(onset, msgAdr="/s_new", msgargs=["s1", 2000, 1, 0, "freq", 300, "amp", 0.5]):
    global clientSC
    bundle = OSC.OSCBundle()
    msg = OSC.OSCMessage()
    print msg
    msg.setAddress(msgAdr)
    msg.extend(msgargs)
    bundle.append(msg)
    bundle.setTimeTag(onset)
    clientSC.send(bundle)


clientMAX = OSC.OSCClient(); clientMAX.connect(("127.0.0.1", 57110)) 
def max_msg(onset, msgAdr="/s_new", msgargs=["freq", 300, "amp", 0.5]):
    global clientMAX
    bundle = OSC.OSCBundle()
    msg = OSC.OSCMessage()
    msg.setAddress(msgAdr)
    msg.extend(msgargs)
    bundle.append(msg)
    bundle.setTimeTag(onset)
    clientMAX.send(bundle)
    
if (soundmodel == "vocal") :    
    sc_msg(0, "/s_new", ["reverb", 1001, 1, 0, "outbus", 0, "room", 0.4, "mix", 0.1, "damp", 0.8]);
else: pass  
# Test sound. 
if (soundmodel == "vocal") :
    # use test tone to check if the reverb is one
    sc_msg(0, "/s_new", ["default", 1002, 1, 1]); now = time.time(); 
    sc_msg(now+0.5, "/n_free", [1002])
else: pass

"""
Mapping here needs some rework. 
"""
# TH: for vocal synth
parspec_vocal = array([ # name, min, max, scaling (lin/exp), default
#("evrate", 0.2, 4, "exp", 0.5, "Hz"),
#("irregularity", 0, 1, "lin", 0, "%"),
("dur", 0.005, 1.5, "exp", 0.4, "secs"), 
("att", 0.001, 0.5, "exp", 0.001, "secs"),
("decslope", -50, 10, "lin", -12, "dB/rm time"),
("amint",  0, 1, "lin", 0, "intensity"),
("amfreq", 1, 50, "exp", 1, "Hz"),
("pitch", 20, 85, "lin", 50, "midinote"),
("chirp", -36, 36, "lin", 0, "semitones/dur"),
("lfnfrq", 5, 50, "exp", 5, "Hz"),
("lfnint", 0, 0.5, "lin", 0, "rel. pitch"),
("vowel", 0, 4, "lin", 2.5, "uoaei"),
("voweldiff", -2.5, 2.5, "lin", 0, "delta"),
("bright", 0.2, 1, "lin", 0.5, "arb.u.")], dtype=[
      ('name', 'S20'), ('min', '>f4'), ('max', '>f4'), ('scaling', 'S10'), ('default', '>f4'), ('unit', 'S20')])

# JJ: New parspec
parspec_abstract = array([ # name, min, max, scaling (lin/exp), default
#("evrate", 0.2, 4, "exp", 0.5, "Hz"),
#("irregularity", 0, 1, "lin", 0, "%"),
("dur", 0., 1., "lin", 0.5, "secs"),
("att", 0., 1., "lin", 0.3, "%"),
("desvol", 0., 1., "lin", 0.5, "dB/dur"),
("pitch", 0, 1., "lin", 0.5, "Hz"),
("chirp", 0., 1., "lin", 0.5, "semitones/dur"),
("lfndepth", 0., 1., "lin", 0., "rate"),
("lfnfreq", 0, 1., "lin", 0., "Hz"),
("amdepth", 0., 1., "lin", 0., "rate"),
("amfreq", 0., 1., "lin", 0., "Hz"),
("richness", 0., 1., "lin", 0.5, "%"),
("lpfreq", 0., 1., "lin", 0.5, "Hz")  # I wonder if this is important.       
    ], dtype=[
      ('name', 'S20'), ('min', '>f4'), ('max', '>f4'), ('scaling', 'S10'), ('default', '>f4'), ('unit', 'S20')])

"""
--------------------------------------------------------------------------------------------------------------
"""

"""
All the GUI functions:
"""
def parmap(par=("pitch", 20, 85, "lin", 50, "midinote"), val=0.5):
    mi, ma = par[1], par[2]
    if(par[3]=="lin"): return mi+(ma-mi)*val
    if(par[3]=="exp"): return mi*exp(log(ma/mi)*val)

def parunmap(par=("pitch", 20, 85, "lin", 50, "midinote"), val=40):
    mi, ma = par[1], par[2]
    if(par[3]=="lin"): return (val-mi)/(ma-mi)
    if(par[3]=="exp"): return log(val/mi)/log(ma/mi)
    
def parvecmap(parspec, vec):
    return array([parmap(parspec[k], v) for k,v in enumerate(vec)])

def parvecunmap(parspec, vec):
    return array([parunmap(parspec[k], v) for k,v in enumerate(vec)])    

# test code:
# print parvecunmap(parspec_vocal, parspec['default']) # get default parameters

def playevent(soundmodel, v):
    # v is unmapped vector, i.e. vector elements in [0,1]
    print v
    if(soundmodel=='vocal'):
        ps = parspec_vocal
        vec = parvecmap(ps, v);
        sc_msg(0, "/s_new", ["jj1", 1002+random.randint(900), 1,1] + 
           [x for pair in zip(ps['name'].tolist(), vec) for x in pair] );
    if(soundmodel=='abstract'):
        ps = parspec_abstract
        vec = parvecmap(ps, v);
        max_msg(0, "/s_new" , [x for pair in zip(ps['name'].tolist(), vec) for x in pair] )
        
    if(soundmodel=='physiological'):
        ps = parspec_physiological
        vec = parvecmap(ps, v);
        max_msg(0, "/s_new" , [x for pair in zip(ps['name'].tolist(), vec) for x in pair] )
        

def mutate(parent, sigma=0.1):
    d=size(parent)
    child = clip((parent + sigma*random.randn(d)), 0, 1)
    return child

def create_next_generation(parentvec, sigma, nr_of_children=4):
    return [parentvec] + [mutate(parentvec, sigma) for k in range(nr_of_children)]

def append_data(dataset, time, target, generation_counter, logsigma, parvec, submit=0, userid=-1, sound="vocal", run=-1):
    dataset.append([userid, sound, run, time, target, generation_counter, logsigma, parvec, submit])    

def savedata(dataset, prefix="user-soundmodel-run-"):
    df = pandas.DataFrame(data, columns=['uid', 'snd', 'run', 'time','target','generation','logsigma','parvec','submit'])
    df.to_csv(prefix + time.strftime("-%Y%m%d-%H%M%S") + ".csv", index=False)

wait for SC to start...
resume: establish OSC interfaces...
 []
 []
 []


## Experiment GUI:

In [3]:
username = "fake"
userid = 1010
soundmodel = "abstract"
run = 1
log_sigma_initval = -1.0
log_sigma_step = -0.2
data = []  # time, target state, generation_counter, sigma, parvec
generation_counter = 0
v = []
target_set_time = time.time()
targetlist = ['happy', 'surprised', 'angry', 'afraid', 'disgusted', 
              'sad', 'calm', 'neutral'];
currentEmotion = 0 # use it to loop through targetlist
degreelist = ['0', '1', '2', '3','4']
class Rating_receiver:
    def __init__(self, ip = '192.168.0.3', port = 8022, num_emotion = 9, num_degree = 4):
        
        self.receive_address = ip, port
        self.num_emotion = num_emotion
        self.num_degree = num_degree
        self.choice = 0
        self.degree = 0
        self.logValue = -1.0
        self.count = 0
        
    def reset_settings(self):
        global generation_counter, v,  soundmodel
        generation_counter = 0
        
        if(soundmodel=='vocal'):    ps = parspec_vocal
        if(soundmodel=='abstract'): ps = parspec_abstract
        if (soundmodel == 'physiological'): ps = parspec_physiological
        v = create_next_generation(parvecunmap(ps, ps['default']), 0.25)
    def spawn(self):
        print"Server Created."
        self.receiveServer = OSC.OSCServer(self.receive_address)
        print self.receive_address# create a serve to receive OSC from the tablet
        self.receiveServer.addDefaultHandlers()
        print self.receiveServer
        print self.receiveServer.address()
        
    def initilisation_handler(self, addr, tags, stuff, source):
        global userid, username, run,soundmodel
        target_set_time = time.time()
        run = int(stuff[3])
        userid,username = stuff[1].split(',')
        soundmodel = stuff[2]
        print userid,username,run,soundmodel
        
    def emotion_handler(self, addr, tags, stuff, source):
        self.choice = targetlist[stuff[0]]
        print self.choice

    def degree_handler(self, addr, tags, stuff, source):
        self.degree = degreelist[stuff[0]]
        

    def log_handler(self, addr, tags, stuff, source):
        self.logValue = float(stuff[-1]/100.0)-5
        print self.logValue

        
        # This is nextVariation from Android
    def next_variation_handler(self, addr, tags, stuff, source):
        global  v,data,soundmodel,run,generation_counter
        generation_counter +=1
        self.degree= int(stuff[1])
        data.append([userid, soundmodel, run, time.time()-target_set_time,targetlist[self.choice], generation_counter, self.logValue, v[int(self.degree)].tolist(), 0])    

        print "variation value ",self.degree
        #append_data(data, time.time() - target_set_time, targetlist[self.choice], generation_counter, 
        #        self.logValue, v[int(self.degree)].tolist(),  userid=userid, sound=soundmodel, run=run)
        v = create_next_generation(v[int(self.degree)], exp(self.logValue))
        self.logValue= float(stuff[0]/100.0)-5        
        
        self.degree = 0
        
    def accept_handler(self, addr, tags, stuff, source):        
        global  v,data,soundmodel,run,generation_counter
        generation_counter +=1
        data.append([userid, soundmodel, run, time.time()-target_set_time,targetlist[self.choice], generation_counter, self.logValue, v[int(self.degree)].tolist(), 1])    
        self.choice= int(stuff[0])
        self.reset_settings()
        self.degree = 0

    def save_handler(self, addr, tags, stuff, source):
        global  v,data, username, run,generation_counter
        print "completed. thanks."
        fname = "../data/%s-%s-run%d" % (username, soundmodel, run) # All 3 needs to be replaced. 
        print fname
        #         savedata(data, fname)
        df = pandas.DataFrame(data, 
                              
            columns=['uid', 'snd', 'run', 'time','target','generation','logsigma','parvec','submit'])
        df.to_csv(fname + time.strftime("-%Y%m%d-%H%M%S") + ".csv", index=False)
        print "data saved to " + fname
        print "Please inform the operator."
        self.reset_settings()
        self.degree = 0
        
    def best_variation_handler(self, addr, tags, stuff, source):
        global v,generation_counter,data, soundmodel
        generation_counter +=1
        parent = int(stuff[0])
        print "parent id",parent# This is the index of the variation. 
        data.append([userid, soundmodel, run, time.time()-target_set_time,targetlist[self.choice], generation_counter, self.logValue, v[int(self.degree)].tolist(), 0])    
        # Play sound
        playevent(soundmodel, v[parent])

        #append_data(data, time.time()-target_set_time, targetlist[self.choice], generation_counter, 
        #        self.logValue, v[parent].tolist(), userid=userid, sound=soundmodel, run=run)
        

    def add_handler(self):
        self.receiveServer.addMsgHandler("/play", self.next_variation_handler)
        self.receiveServer.addMsgHandler("/next", self.accept_handler)
        self.receiveServer.addMsgHandler("/emo", self.emotion_handler)
        self.receiveServer.addMsgHandler("/degree", self.degree_handler)
        self.receiveServer.addMsgHandler("/save", self.save_handler)
        self.receiveServer.addMsgHandler("/init", self.initilisation_handler)
        self.receiveServer.addMsgHandler("/logChange", self.log_handler)
        self.receiveServer.addMsgHandler("/variationselection", self.best_variation_handler)
            
    def print_registered_func(self):
        for addr in self.receiveServer.getOSCAddressSpace():
            print addr
            
    def start(self):
        # Start OSCServer
        print "\nStarting OSCServer."
        self.emorating_oscServer = threading.Thread(target = self.receiveServer.serve_forever)
        self.emorating_oscServer.start()
        print "\nOSCServer established."
        
    def stop(self):
        # Close the OSC server
        print "\nClosing OSCServer."
        self.receiveServer.close()
        print "Waiting for Server-thread to finish"
        try:
            self.emorating_oscServer.join() ##!!!
            print "Done"
        except AttributeError:
            print AttributeError

In [4]:
ipAddr = socket.gethostbyname(socket.getfqdn())
init_receiver = Rating_receiver (ip = ipAddr, port = 8022,num_emotion = 8, num_degree = 4)
init_receiver.spawn()
init_receiver.add_handler()
init_receiver.print_registered_func()
init_receiver.reset_settings()
init_receiver.start()

Server Created.
('129.70.149.85', 8022)
OSCServer v0.3.5b-5294 listening on osc://ip-129-70-149-85.wlan.dyn.cit-ec.net:8022
('129.70.149.85', 8022)
/variationselection
/next
/print
default
/init
/info
/play
/emo
/error
/logChange
/degree
/save

Starting OSCServer.

OSCServer established.
parent id 4
[ 0.74569542  0.52498351  0.56210307  0.57640219  0.14753624  0.46049322
  0.35980553  0.          0.33127883  0.74399448  0.57017475]


In [18]:
try:
    init_receiver.stop()
    rating_receiver.stop()
except:
    print "OK"


Closing OSCServer.
Waiting for Server-thread to finish
Done
OK


Exception in thread Thread-8:
Traceback (most recent call last):
  File "/Users/sozen/anaconda/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/Users/sozen/anaconda/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "OSC.py", line 1826, in serve_forever
    self.handle_request()	# this times-out when no data arrives.
  File "/Users/sozen/anaconda/lib/python2.7/SocketServer.py", line 271, in handle_request
    fd_sets = _eintr_retry(select.select, [self], [], [], timeout)
  File "/Users/sozen/anaconda/lib/python2.7/SocketServer.py", line 150, in _eintr_retry
    return func(*args)
error: (9, 'Bad file descriptor')



## 8. Load all data for evaluation

In [ ]:
import glob
import pandas
def parselist(s):
    return array([ float(el) for el in s.translate(None, "[]").split(",") ], dtype='float64')

converterdict = {'parvec': parselist};

# load all data files of pattern into dataframe
for i, fname in enumerate(glob.glob("../data/*.csv")):
    print fname
    df = pandas.read_csv(fname, converters=converterdict)
    if(i==0): 
        da = df
    else: 
        da = da.append(df)

In [ ]:
da

## 9. extract all optimization endpoints and play them in a series
(can now be done easier since I added logging for the submit button... to be updated)

In [ ]:
# play all sounds for a given emtion
# soundmodel = 'vocal'
dsel = da[(da['submit']==1) & (da['target']=='disgusted') & (da['snd']==soundmodel)]

for el in dsel.iterrows():
    parvec = el[1]['parvec']
    print el[1]['uid']
    playevent(soundmodel, parvec)
    time.sleep(1.0)    

In [ ]:
# play center of mass of all emotion-assigned samples
targets = ['happy', 'surprised', 'angry', 'disgusted', 'sad', 'calm']
soundmodel = "vocal"
for t in targets:
    print t
    dsel = da[(da['submit']==1) & (da['target']==t) & (da['snd']==soundmodel)]
    print dsel
    vt = mean(dsel['parvec']) 
    print vt
    playevent(soundmodel, vt)
    time.sleep(1.5)    

## 10. interpolation experiment: play series of sounds between prototypes

In [ ]:
# we need a function to convert a array of arrays into a regular 2D array
def aoa_2d_array(pv):
    r = zeros((size(pv), size(pv[0])))
    for i, el in enumerate(pv): r[i] = el
    return r 
# test code
#soundmodel = 'abstract'
#aoa_2d_array(da[(da['submit']==1) & (da['snd']==soundmodel)].parvec.values)

In [ ]:
# array of all submitted parameter vectors
# soundmodel='vocal'
dtmp = da[(da['submit']==1) & (da['snd']==soundmodel) & 
          (da['uid']==1003) & (da['run']==0)].parvec.values
pvarr = aoa_2d_array(dtmp)
playevent(soundmodel, mean(pvarr, 0)) # mean vector of all submitted prototypes

In [ ]:
dtmp = da[(da['submit']==1) & (da['snd']==soundmodel) & (da['uid']==1001) & (da['run']==0) ]
# print dtmp
par = aoa_2d_array(dtmp['parvec'].values)
# interpolate between emotional prototypes
for l in arange(0, 1, 0.1):
    playevent(soundmodel, l*par[2]+(1.0-l)*par[3])
    time.sleep(1)

## Kernel regression-mapping for navigating between Emotionals Prototypes

In [ ]:
# assemble 6 prototype vectors
dsel = da[(da['submit']==1) & (da['uid']==1003) & (da['snd']=='abstract') & (da['run']==1)]

In [ ]:
da

In [ ]:
targets = ['happy', 'surprised', 'angry', 'disgusted', 'sad', 'calm']
pvec = dsel[:1].parvec.values[0]
dim = len(pvec)
Nrows = len(targets)
pvecs = zeros((Nrows, dim))
for i, t in enumerate(targets):
    print i, t
    pvecs[i]= dsel[dsel['target']==t].parvec.values[0]
    #print t; playevent("jj1", parspec, pvecs[i]); time.sleep(1.5)

# kernel regression: input positions
xvecs = zeros((Nrows, 2))
for i in range(len(targets)):
    xvecs[i] = [cos(2*pi*i/Nrows+0.1), sin(2*pi*i/Nrows+0.1)]

def kernel(x, y, sigma=1):
    return exp(-0.5*sum((x-y)**2)/sigma**2)

def krm(xvecs, pvecs, xvec, sigma=1):
    n=shape(xvecs)[0]
    nom = zeros(dim)
    den = 0
    for i in range(n):
        temp = kernel(xvecs[i], xvec, sigma)
        print temp
        nom += temp*pvecs[i]
        den += temp
    return nom/den
# print xvecs[0], krm(xvecs, pvecs, xvecs[0], sigma=0.1)    
# playevent(soundmodel, krm(xvecs, pvecs, array([-1,1]), sigma=0.9))

In [ ]:
pvecs

In [ ]:

import matplotlib.pyplot as plt


fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(xvecs[:,0], xvecs[:,1], "o", markersize=15)
ax.set_aspect('equal', 'datalim')

wkrmsigma = widgets.FloatSlider(value=0.2, min=0, max=1, step=0.01, description='krm-sigma:')

def onclick(event, verbose=False):
    x = event.xdata; y = event.ydata;
    if(event.button==1): #left mouse click
        if(verbose):
            print('(%f, %f)' % (x, y))
        playevent(soundmodel, krm(xvecs, pvecs, array([x,y]), sigma=wkrmsigma.value))
cid = fig.canvas.mpl_connect('button_press_event', onclick)
display(wkrmsigma)

In [ ]:
soundmodel

In [ ]:
# Set up OSC receiver for the tablet interaction. 
soundmodel = 'vocal'
129-70-149-109
receive_address = "129.70.149.122", 50010 # The receive address needs to be controlled by the tablet
receiveSever = OSC.OSCServer(receive_address) # create a serve to receive OSC from the tablet
receiveSever.addDefaultHandlers()
x = 0.0
y = 0.0
wkrmsigma = 0.8 # later it can be control by tablet. 

# Need to change it to a class. 

def update_OSCAddress(addr,tags,stuff, source):
    global receive_address
    receive_address = stuff[0], int(stuff[1])
    print "New address:" + stuff[0]
    print "New port: "+ stuff[1]
    


def printing_handler(addr, tags, stuff, source):
    global x, y, soundmodel, xvecs, pvecs, wkrmsigma
    x = float(stuff[0])
    y = float(stuff[1])
    print x, y
    playevent(soundmodel, krm(xvecs, pvecs, array([x,y]), sigma=wkrmsigma))

receiveSever.addMsgHandler("/a", printing_handler) # adding our function
receiveSever.addMsgHandler("/addrInfo", update_OSCAddress)
# just checking which handlers we have added
print "Registered Callback-functions are :"
for addr in receiveSever.getOSCAddressSpace():
    print addr
    
# Start OSCServer
print "\nStarting OSCServer."
st_oscSever = threading.Thread( target = receiveSever.serve_forever )
st_oscSever.start()

In [ ]:
# Close the OSC server
print "\nClosing OSCServer."
receiveSever.close()
print "Waiting for Server-thread to finish"
st_oscSever.join() ##!!!
print "Done"